In [1]:
import talib
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
def add_fractal(df, i, look_back=2):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan


def add_beep_boop(row):
  macdhist, ema50, mid_low, mid_high = row[['macdhist', 'ema50', 'Mid_Low', 'Mid_High']]

  if float(macdhist) > 0 and float(mid_low) > float(ema50):
    return 1

  elif float(macdhist) < 0 and float(mid_high) < float(ema50):
    return 2

  else:
    return 0


# def add_trade(df, i, atr_percentage=0.2, bar_length=0.0005):
#     if i < 3:
#         return np.nan

#     ema200 = df.loc[df.index[i - 1], 'ema200']
#     beep_boop1 = df.loc[df.index[i - 1], 'beep_boop']
#     beep_boop2 = df.loc[df.index[i - 2], 'beep_boop']
#     beep_boop3 = df.loc[df.index[i - 3], 'beep_boop']
#     atr = df.loc[df.index[i - 1], 'atr']
#     curr_ao = df.loc[df.index[i], 'Ask_Open']
#     curr_bo = df.loc[df.index[i], 'Bid_Open']
#     spread = abs(curr_ao - curr_bo)
#     enough_volatility = (spread / atr) <= atr_percentage
#     curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
#     curr_mh = df.loc[df.index[i - 1], 'Mid_High']
#     curr_bar_length = abs(curr_mh - curr_ml)
#     macd2, macdsignal2, macdhist2 = df.loc[df.index[i - 2], ['macd', 'macdsignal', 'macdhist']]
#     macd1, macdsignal1, macdhist1 = df.loc[df.index[i - 1], ['macd', 'macdsignal', 'macdhist']]

#     if beep_boop3 == 1 and beep_boop2 == 1 and beep_boop1 == 1 and curr_ml > ema200 and enough_volatility and curr_bar_length <= bar_length:
#         return 1

#     elif beep_boop3 == 2 and beep_boop2 == 2 and beep_boop1 == 2 and curr_mh < ema200 and enough_volatility and curr_bar_length <= bar_length:
#         return 2

#     else:
#         return 0

def add_trade(df, i, adx_cutoff=20, spread_cutoff=0.0002, bar_length=0.0005):
    if i < 2:
        return np.nan

    ema200 = df.loc[df.index[i - 1], 'ema200']
    adx = df.loc[df.index[i - 1], 'adx']
    curr_ao = df.loc[df.index[i], 'Ask_Open']
    curr_bo = df.loc[df.index[i], 'Bid_Open']
    spread = abs(curr_ao - curr_bo)
    enough_volatility = spread <= spread_cutoff
    curr_ml2 = df.loc[df.index[i - 2], 'Mid_Low']
    curr_ml1 = df.loc[df.index[i - 1], 'Mid_Low']
    curr_mh2 = df.loc[df.index[i - 2], 'Mid_High']
    curr_mh1 = df.loc[df.index[i - 1], 'Mid_High']
    curr_bar_length = abs(curr_mh1 - curr_ml1)
    macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
    macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
    macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
    trending = adx >= adx_cutoff

    if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml2 > ema200 and curr_ml1 > ema200 and max(macd_vals) == 0 and enough_volatility and trending and curr_bar_length <= bar_length:
        return 1

    elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh2 < ema200 and curr_mh1 < ema200 and min(macd_vals) == 0 and enough_volatility and trending and curr_bar_length <= bar_length:    
        return 2

    else:
        return 0

In [5]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['beep_boop'] = df.apply(add_beep_boop, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['trade'] = [add_trade(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['beep_boop'].value_counts())
print(df['fractal'].value_counts())
print(df['trade'].value_counts())

0    35299
1    20429
2    18333
Name: beep_boop, dtype: int64
0.0    55661
1.0     9386
2.0     9014
Name: fractal, dtype: int64
0.0    73905
1.0       88
2.0       68
Name: trade, dtype: int64


In [6]:
df['atr'].mean()

0.0005531407860600905

In [6]:
# df_fractals = df[df.fractal != 0]
# df_fractal_prices = [
#     df_fractals.loc[df_fractals.index[i], 'Mid_Low'] if df_fractals.loc[df_fractals.index[i], 'fractal'] == 1 else df_fractals.loc[
#         df_fractals.index[i], 'Mid_High'] for i in range(len(df_fractals))]

In [15]:
def plot_candles(df_plot):
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.Date, open=df_plot.Mid_Open, high=df_plot.Mid_High, low=df_plot.Mid_Low, close=df_plot.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',
        decreasing_line_color='#FF3A4C'
    ))

    df_fractals = df_plot[df_plot.fractal != 0]
    df_fractal_prices = [df_fractals.loc[df_fractals.index[i], 'Mid_Low'] if df_fractals.loc[df_fractals.index[i], 'fractal'] == 1 else df_fractals.loc[df_fractals.index[i], 'Mid_High'] for i in range(len(df_fractals))]

    fig.add_trace(go.Scatter(
        x=df_fractals.Date,
        y=df_fractal_prices,
        marker=dict(color="crimson", size=10),
        name='Fractal',
        mode='markers'
        ))

    fig.add_trace(go.Scatter(
        x=df_plot.Date,
        y=df_plot['ema200'],
        marker=dict(color="yellow", size=10),
        name='EMA 200',
        mode='lines'
        ))

    df_buys = df_plot[df_plot.trade == 1]
    df_sells = df_plot[df_plot.trade == 2]

    fig.add_trace(go.Candlestick(
        x=df_buys.Date, open=df_buys.Mid_Open, high=df_buys.Mid_High, low=df_buys.Mid_Low, close=df_buys.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='blue',
        decreasing_fillcolor='blue',
        increasing_line_color='blue',
        decreasing_line_color='blue',
        name='Buy'
    ))

    fig.add_trace(go.Candlestick(
        x=df_sells.Date, open=df_sells.Mid_Open, high=df_sells.Mid_High, low=df_sells.Mid_Low, close=df_sells.Mid_Close,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='orange',
        decreasing_fillcolor='orange',
        increasing_line_color='orange',
        decreasing_line_color='orange',
        name='Sell'
    ))

    fig.update_layout(width=1400,height=600,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")

    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )

    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )

    fig.show()

In [56]:
df[df.trade == 2].index

Int64Index([ 1512,  4071,  4538,  4856,  4863,  6596,  7558, 10560, 11612,
            17442, 18579, 18939, 21193, 21488, 21496, 21563, 23457, 26111,
            27000, 27456, 30401, 32443, 35697, 36554, 37498, 39647, 40692,
            42475, 43094, 43919, 44234, 46812, 47087, 47103, 47381, 47741,
            47915, 48798, 50445, 53927, 53939, 53958, 57936, 57972, 60112,
            60456, 60863, 61965, 62487, 62523, 63242, 63695, 63699, 63754,
            64239, 64242, 64381, 64716, 65142, 65332, 66762, 67470, 69056,
            69169, 70576, 70581, 71130, 71934],
           dtype='int64')

In [59]:
plot_candles(df.iloc[6578:6700, :])